<a href="https://colab.research.google.com/github/khyatimirani/AI-Experiments/blob/main/HealthModelFineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

!pip install -q unsloth
# Also get the latest nightly Unsloth!
!pip install -q --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
!pip install -q unsloth


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.7/405.7 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 961.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 132.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! Unsloth also supports RoPE (Rotary Positinal Embedding) scaling internally.
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit, # Will load the 4Bit Quantized Model
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.4: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16, # a higher alpha value assigns more weight to the LoRA activations
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2026.1.4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
from datasets import load_dataset

dataset = load_dataset('altaidevorg/women-health-mini', split = "train")

README.md: 0.00B [00:00, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


women-health-mini.jsonl:   0%|          | 0.00/35.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10348 [00:00<?, ? examples/s]

In [ ]:
print(dataset[:5])

{'conversations': [[{'role': 'user', 'content': "I had an unsafe abortion and now I'm experiencing complications. Can you help me?"}, {'role': 'assistant', 'content': "I understand you're going through a very difficult time, and I want to assure you that I'm here to help.  Having an unsafe abortion can lead to serious complications, and it's crucial to address them immediately.  \n\nThe potential complications following an unsafe abortion are significant and can range from mild to life-threatening. These can include:\n\n* **Incomplete abortion:** This means some pregnancy tissue remains in the uterus, leading to continued bleeding, pain, and infection.\n* **Infection:**  This is a major concern and can manifest as fever, chills, severe abdominal pain, foul-smelling vaginal discharge, and weakness.  Untreated infection can spread to the bloodstream (septicemia), leading to severe illness and even death.\n* **Excessive bleeding (hemorrhage):** This is a life-threatening emergency and can

In [ ]:
fertility_prompt = """You are an AI Fertility Companion designed to support patients undergoing fertility treatments such as IVF, IUI, and related procedures.

Your responsibilities:
- Respond in a calm, empathetic, and reassuring tone
- Follow doctor-approved medical guidance strictly
- Provide clear, patient-friendly explanations
- Normalize common symptoms without dismissing concerns
- Clearly mention when a patient should contact their clinic
- Do NOT provide diagnosis or replace a doctor
- Do NOT reveal internal reasoning or thought processes
- Avoid alarming or absolute language

<patient_question>
{}
</patient_question>

<doctor_approved_guidance>
{}
</doctor_approved_guidance>

<ideal_assistant_response>
{}
</ideal_assistant_response>
"""
EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    all_formatted_texts = []

    # Determine if 'examples' contains a single conversation or a batch of conversations
    # Check the type of the first element in examples["conversations"]
    if isinstance(examples["conversations"][0], dict):
        # This is a single example, so examples["conversations"] is directly the list of turns
        list_of_conversation_lists = [examples["conversations"]]
    else:
        # This is a batch, so examples["conversations"] is a list of lists of turns
        list_of_conversation_lists = examples["conversations"]

    for conversation_list in list_of_conversation_lists:
        problem = ""
        solution = ""
        for turn in conversation_list:
            if turn["role"] == "user":
                problem = turn["content"]
                break
        for turn in conversation_list:
            if turn["role"] == "assistant":
                solution = turn["content"]
                break

        guide = ""
        formatted_text = fertility_prompt.format(
            problem.strip(),
            guide.strip(),
            solution.strip()
        ) + EOS_TOKEN
        all_formatted_texts.append(formatted_text)

    return all_formatted_texts

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    # dataset_text_field = "text", # Removed as formatting_func will directly return processed strings
    max_seq_length = max_seq_length,
    dataset_num_proc = 2, # Number of processors to use for processing the dataset
    packing = False, # Can make training 5x faster for short sequences.
    formatting_func = formatting_prompts_func,
    args = TrainingArguments(
        per_device_train_batch_size = 2, # The batch size per GPU/TPU core
        gradient_accumulation_steps = 4, # Number of steps to perform befor each gradient accumulation
        warmup_steps = 5, # Few updates with low learning rate before actual training
        max_steps = 60, # Specifies the total number of training steps (batches) to run.
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit", # Optimizer
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc for observability
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/10348 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10,348 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 12,156,928 of 3,224,906,752 (0.38% trained)


Step,Training Loss
1,1.683100
2,1.693400
3,1.658900
4,1.567400
5,1.531100
6,1.421200
7,1.540000
8,1.448100
9,1.357300
10,1.279300


In [ ]:
from unsloth.chat_templates import get_chat_template
sys_prompt = """You are an AI Fertility Companion designed to support patients undergoing fertility treatments such as IVF, IUI, and related procedures.

Your responsibilities:
- Respond in a calm, empathetic, and reassuring tone
- Follow doctor-approved medical guidance strictly
- Provide clear, patient-friendly explanations
- Normalize common symptoms without dismissing concerns
- Clearly mention when a patient should contact their clinic
- Do NOT provide diagnosis or replace a doctor
- Do NOT reveal internal reasoning or thought processes
- Avoid alarming or absolute language

<patient_question>
{}
</patient_question>

<doctor_approved_guidance>
{}
</doctor_approved_guidance>

<ideal_assistant_response>
{}
</ideal_assistant_response>
"""
# Corrected: Provide empty strings for the other two placeholders
message = sys_prompt.format("I often have severe cramps during my periods that interfere with daily activities. Is this something that should be evaluated?", "", "")
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": message},
]
processed_inputs_tensor = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
)

# Fix: processed_inputs_tensor is directly the input_ids tensor
inputs = processed_inputs_tensor.to("cuda")
# Manually create attention_mask as it's not returned by apply_chat_template in this context
attention_mask = torch.ones_like(inputs).to("cuda")

outputs = model.generate(input_ids = inputs, attention_mask = attention_mask, max_new_tokens = 1024, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
response = tokenizer.batch_decode(outputs)
print("=========== LLM Responded")
print(response)

=========== LLM Responded
["<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nYou are an AI Fertility Companion designed to support patients undergoing fertility treatments such as IVF, IUI, and related procedures.\n\nYour responsibilities:\n- Respond in a calm, empathetic, and reassuring tone\n- Follow doctor-approved medical guidance strictly\n- Provide clear, patient-friendly explanations\n- Normalize common symptoms without dismissing concerns\n- Clearly mention when a patient should contact their clinic\n- Do NOT provide diagnosis or replace a doctor\n- Do NOT reveal internal reasoning or thought processes\n- Avoid alarming or absolute language\n\n<patient_question>\nI often have severe cramps during my periods that interfere with daily activities. Is this something that should be evaluated?\n</patient_question>\n\n<doctor_approved_guidance>\n\n</do